# MQT Decision Diagram backend benchmark

This notebook runs a small selection of example circuits solely on the `MQT Decision Diagram` simulator. It demonstrates how to use `BenchmarkRunner` for collecting timing and memory metrics. Extend the `circuits_to_run` list or adjust parameters to analyze additional workloads.


In [1]:
from quasar.backends import DecisionDiagramBackend
from benchmarks.runner import BenchmarkRunner
from benchmarks import circuits
import pandas as pd

In [2]:
# Select representative circuits
circuits_to_run = [
    ("GHZ", circuits.ghz_circuit(5)),
    ("QFT", circuits.qft_circuit(5)),
    ("Grover", circuits.grover_circuit(5, 1)),
]

runner = BenchmarkRunner()
backend = DecisionDiagramBackend()
for name, circ in circuits_to_run:
    res = runner.run_multiple(circ, backend, return_state=False, repetitions=3)
    res["circuit"] = name

df = runner.dataframe()
df[["circuit", "prepare_time_mean", "run_time_mean", "total_time_mean", "prepare_peak_memory_mean", "run_peak_memory_mean"]]


,circuit,prepare_time_mean,run_time_mean,total_time_mean,prepare_peak_memory_mean,run_peak_memory_mean
0,GHZ,0.026385,0.000368,0.026753,240.000000,2777.000000
1,QFT,0.006584,0.000616,0.007200,288.000000,4475.000000
2,Grover,0.001611,0.005815,0.007425,4893.333333,26395.666667


In [3]:
import pandas as pd
from benchmarks.stats_utils import stats_table

def add_stats(df, quasar_col='QuASAr', baseline_cols=None, test='ttest', correction='bonferroni'):
    """Compute statistics comparing QuASAr with baselines.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with per-circuit results. One column must correspond to QuASAr,
        others to baselines.
    quasar_col : str
        Name of the column containing QuASAr results.
    baseline_cols : list[str] | None
        Columns to treat as baselines. Defaults to all columns except quasar_col.
    test : str
        'ttest' or 'wilcoxon'.
    correction : str
        'bonferroni' or 'fdr_bh'.

    Returns
    -------
    pd.DataFrame
        Table with baseline name, statistic, corrected p-value, and effect size.
    """
    if baseline_cols is None:
        baseline_cols = [c for c in df.columns if c != quasar_col]
    baselines = {c: df[c] for c in baseline_cols}
    return stats_table(df[quasar_col], baselines, test=test, correction=correction)

# Example usage after computing results DataFrame named `results_df`:
# stats_df = add_stats(results_df)
# stats_df


In [4]:

# Record parameters and results
import json, pathlib
try:
    import ipynbname
    nb_name = ipynbname.path().stem
except Exception:  # pragma: no cover
    nb_name = 'notebook'

# Collect simple parameters from globals
_params = {
    k: v for k, v in globals().items()
    if not k.startswith('_') and isinstance(v, (int, float, str, bool, list, dict, tuple))
}
pathlib.Path('../results').mkdir(exist_ok=True)
with open(f"../results/{nb_name}_params.json", 'w') as f:
    json.dump(_params, f, indent=2, default=str)
if 'results' in globals():
    try:
        with open(f"../results/{nb_name}_results.json", 'w') as f:
            json.dump(results, f, indent=2, default=str)
    except TypeError:
        pass
print(json.dumps(_params, indent=2, default=str))


{
  "In": [
    "",
    "from quasar.backends import DecisionDiagramBackend\nfrom benchmarks.runner import BenchmarkRunner\nfrom benchmarks import circuits\nimport pandas as pd",
    "# Select representative circuits\ncircuits_to_run = [\n    (\"GHZ\", circuits.ghz_circuit(5)),\n    (\"QFT\", circuits.qft_circuit(5)),\n    (\"Grover\", circuits.grover_circuit(5, 1)),\n]\n\nrunner = BenchmarkRunner()\nbackend = DecisionDiagramBackend()\nfor name, circ in circuits_to_run:\n    res = runner.run_multiple(circ, backend, return_state=False, repetitions=3)\n    res[\"circuit\"] = name\n\ndf = runner.dataframe()\ndf[[\"circuit\", \"prepare_time_mean\", \"run_time_mean\", \"total_time_mean\", \"prepare_peak_memory_mean\", \"run_peak_memory_mean\"]]",
    "import pandas as pd\nfrom benchmarks.stats_utils import stats_table\n\ndef add_stats(df, quasar_col='QuASAr', baseline_cols=None, test='ttest', correction='bonferroni'):\n    \"\"\"Compute statistics comparing QuASAr with baselines.\n\n    Par